# Luxurious git branch switch with fzf

Put the following in your `.bashrc`:

```bash
# This function must return either upstream/main or upstream/master,
# depending on which one is present. If both are present it should
# return upstream/main. If neither are present it should return
# origin/main or origin/master, depending on which is present.
function test_remote_branch_exists() {
    local branch_name="$1"
    git branch --remote --list "$branch_name" | grep -q "$branch_name" && echo "$branch_name" || echo ""
}

export -f test_remote_branch_exists


function get_primary_remote() {
    local upstream_main_exists=$(test_remote_branch_exists "upstream/main")
    if [ -n "$upstream_main_exists" ]; then
        echo "upstream/main"
        return
    fi
    upstream_master_exists=$(test_remote_branch_exists "upstream/master")
    if [ -n "$upstream_master_exists" ]; then
        echo "upstream/master"
        return
    fi
    origin_main_exists=$(test_remote_branch_exists "origin/main")
    if [ -n "$origin_main_exists" ]; then
        echo "origin/main"
        return
    fi
    origin_master_exists=$(test_remote_branch_exists "origin/master")
    if [ -n "$origin_master_exists" ]; then
        echo "origin/master"
        return
    fi
}

export -f get_primary_remote

function show_branch_diff() {
    local target_branch="$1"
    git diff --color=always "$(base_commit "$target_branch")" "$target_branch"
}

export -f show_branch_diff

function base_commit() {
    local target_branch="$1"
    git merge-base "$(get_primary_remote)" "$target_branch"
}
export -f base_commit

# Have to add the branch here
# alias gcb='git branch --sort=-committerdate | fzf | xargs git checkout'
alias gcb='git branch | fzf --preview "show_branch_diff {-1}" | cut -c 3- | xargs git checkout'
```

Now, the `gcb` command will show you a fuzzy finder with diff preview on all your git branches